# Phuture's Liquidity Mining Calculations - How much should we distribute?



First we import the coingecko API to retrieve the ETH price, set the token0 and token1 balances of the ETH/PHTR liquidity pool and set up our variables.

In [49]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

x=21849238949042713374/10**18 #ETH
y=392564217471366977912041/10**18  #PHTR
k=x*y
target_investment_size = 10 #Denominated in ETH
target_price_impact = -0.5 #Adjust to target a different price impact. Insert as a percentage without the % sign
eth_price = cg.get_price('ethereum', vs_currencies = 'usd')['ethereum']['usd'] #Current ETH price
phtr_price = cg.get_price('phuture',vs_currencies = 'usd')['phuture']['usd'] #USD denominated PHTR price
target_apr = 0.10 #Target APY for liquidity mining program once pool value required to satisfy slippage criterion has been reached

Define a function to calculate the spot price for an infinitesimally small trade size.

In [50]:
def spot_rate(x,y):
    return y/x

Define a function to calculate the actual trade price based on the target_investment_size.

In [51]:
def trade_price(x, y):
    new_x = target_investment_size+x
    new_y = (x*y)/new_x
    y_out = y-new_y
    return y_out/target_investment_size

Define a function to calculate the amount of slippage in percentatage terms generated from a trade of size equal to the target_investment_size. This function will be used in the while loop so we will prefix it with an underscore to delineate it as internal.

In [52]:
def _slippage(x, y):
    abs = spot_rate(x,y) - trade_price(x,y)
    percent = (trade_price(x,y)/spot_rate(x,y) -1)*100
    return percent

Define an external slippage function with print statements.

In [53]:
def slippage(x,y):
    abs = spot_rate(x,y) - trade_price(x,y)
    percent = (trade_price(x,y)/spot_rate(x,y) -1)*100
    print('Spot Price:',spot_rate(x,y),'PHTR per 1 ETH')
    print('Trade Price:',trade_price(x,y),'PHTR per 1 ETH')
    print('Slippage: '+str(percent) +'%')
    return percent

Define a function that allows us to change the price of PHTR. This helps us to understand how the required amount of PHTR rewards change dependning on the token price. 

In [54]:
def adjust_price(k,new_price=x/y):
    global x
    global y
    global phtr_price
    x= (k*new_price)**(1/2)
    y = (k/new_price)**(1/2)
    phtr_price = x/y*eth_price
    print(phtr_price)

We then adjust the price to approx 0.4USD. This is to factor in some growth in the price of PHTR. The accuracy of this price is only some what relevant. The LM program is split into two 6 month tranches, so if the price is lagging this estimate we can bolster rewards in the second 6 month tranche.

In [55]:
adjust_price(k,0.0001872659176) #Insert a new ETH denominated price to see how this affects the required PHTR to satisfy the target yield. The default is current price.

0.39975842695989605


Create a while loop that increments the balance of ETH and PHTR in the pool until a minimum slippage requirement is met. Here we are using our variable target_price_impact and as mentioned before, slippage is calculated on our target_investment_size of 10 eth. 

In [56]:
while _slippage(x,y) < target_price_impact:
    x+=1
    y+=spot_rate(x,y)

After the while loop has completed we can calculate the USD balance of the pool, then based off of this, calculate the required USD to satisfy our target_apr and finally calculate the amount of PHTR that needs to be distributed. 

In [57]:
total_usd_value = x*eth_price+y*phtr_price
usd_required = total_usd_value * target_apr
phtr_required = usd_required/phtr_price

Lastly, the print statements that show the pertinent output information.

In [58]:
print('Reserves:','x:',x,'y:',y)
slippage(x,y)
print('PHTR Price:',phtr_price,'USD')
print('Total USD value of pool required to satisfy slippage criterion:',total_usd_value)
print('Required USD to satisfy target yield:',usd_required)
print('Required PHTR to satisfy target yield:',phtr_required)

Reserves: x: 1990.0777086702124 y: 10373519.812542059
Spot Price: 5212.620475747019 PHTR per 1 ETH
Trade Price: 5186.558385993354 PHTR per 1 ETH
Slippage: -0.4999805735891316%
PHTR Price: 0.39975842695989605 USD
Total USD value of pool required to satisfy slippage criterion: 8395140.747774519
Required USD to satisfy target yield: 839514.0747774519
Required PHTR to satisfy target yield: 2100053.477701103


To summarise, the ETH/PHTR pool would need to have 1990.0777086702124 ETH and 10,373,519.8 PHTR in order to produce less than 0.5% slippage on a 10 ETH trade. In order to provide a 10% APR on the 8,395,140.8 USD pool size we would need to distribute 2,100,053.5 PHTR.